# Detecção de Fraude em Cartões de Crédito com Validação Cruzada (5 Folds)

Neste notebook, utilizamos o dataset de Fraude em Cartões de Crédito para treinar um modelo de rede neural utilizando o `MLPClassifier` do scikit-learn. O fluxo do notebook inclui:
- Carregamento do dataset
- Preparação dos dados
- Configuração da validação cruzada
- Treinamento e avaliação do modelo em 5 folds
- Agregação dos resultados

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Carregamento do Dataset

O dataset de Fraude em Cartões de Crédito (por exemplo, `creditcard.csv`) deve estar na mesma pasta deste notebook. Ele contém informações sobre transações, onde a coluna `Class` indica se a transação é fraudulenta (1) ou legítima (0).

In [ ]:
# Carrega o dataset de fraude em cartões de crédito
df = pd.read_csv('creditcard.csv')

# Visualiza as primeiras linhas do dataset
print("Visualização do dataset:")
print(df.head())

## Preparação dos Dados

Separamos as features (todas as colunas, exceto `Class`) da variável alvo e verificamos a distribuição das classes para entender o desbalanceamento do dataset.

In [ ]:
# Separa as features e a variável alvo
X = df.drop(columns=['Class']).values
y = df['Class'].values

# Exibe a distribuição das classes
print("\nDistribuição das classes:")
print(pd.Series(y).value_counts())

## Configuração da Validação Cruzada com 5 Folds

Utilizamos o `StratifiedKFold` para manter a proporção das classes em cada fold. Para cada um dos 5 folds, realizamos os seguintes passos:
1. Dividimos os dados em conjunto de treino e teste para o fold atual.
2. Aplicamos a padronização dos dados com `StandardScaler` (ajustado apenas no conjunto de treino).
3. Treinamos o modelo `MLPClassifier` com duas camadas ocultas.
4. Avaliamos o modelo e armazenamos a acurácia de cada fold.

In [ ]:
# Configura a validação cruzada com 5 folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # O que significam os argumentos dessa classe

# Lista para armazenar as acurácias de cada fold
acc_scores = []

fold = 1
for train_index, test_index in skf.split(X, y):
    print(f"\n=== Fold {fold} ===")
    
    # Separação dos dados para o fold atual
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    # Padronização dos dados utilizando StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold) #por que fazer o fit em cada iteração?
    X_test_scaled = scaler.transform(X_test_fold)
    
    # Definição e treinamento do modelo MLPClassifier
    model = MLPClassifier(hidden_layer_sizes=(64, 32), #Essa quantidade de neuronios está apropriada?
                          activation='relu', # O que significa este argumento?
                          solver='adam',
                          max_iter=200,
                          random_state=42,
                          early_stopping=True)
    
    model.fit(X_train_scaled, y_train_fold)
    
    # Avaliação do modelo para o fold atual
    y_pred_fold = model.predict(X_test_scaled)
    acc = accuracy_score(y_test_fold, y_pred_fold)
    acc_scores.append(acc) # Por que colocar a acurácia numa lista?
    
    print(f"Acurácia no Fold {fold}: {acc:.4f}")
    print(f"Relatório de Classificação do Fold {fold}:\n{classification_report(y_test_fold, y_pred_fold)}")
    
    fold += 1

## Resultados Agregados

Após a validação cruzada, calculamos a acurácia média dos 5 folds para ter uma visão geral do desempenho do modelo.

In [ ]:
# Calcula a acurácia média
mean_accuracy = np.mean(acc_scores) # por que calcular a acurácia média
# Onde está o modelo que chega nesta acurácia?
print("========================================")
print(f"Acurácias de cada fold: {acc_scores}")
print(f"Acurácia média nos 5 folds: {mean_accuracy:.4f}")

### Itere sobre esse experimento para melhorar a sua acurácia

In [ ]:
#Teste várias variações dos hyperparâmetros